**k-平均法**

1. 人間側がクラスター(グループ)の数をきめる。

2. ランダムに振られた点(重心)から近いものをクラスターとする。

3. 紐づいたクラスターとの距離をもとに重心を移動させる。



In [68]:
import pandas as pd
import numpy as np

In [69]:
df = pd.read_csv("convinience_store.csv")
df.head()

,No,弁当・麺類,飲料,おにぎり・サンドイッチ,スイーツ,カップスープ,サラダ
0,1,25350,3650,8945,0,4867,8945
1,2,24500,0,0,1827,0,0
2,3,23050,5750,11570,0,7667,11570
3,4,22850,4100,10145,0,5467,10145
4,5,22500,0,0,0,0,0


In [70]:
x=df.drop('No', axis=1).values
x[:3]

array([[25350,  3650,  8945,     0,  4867,  8945],
       [24500,     0,     0,  1827,     0,     0],
       [23050,  5750, 11570,     0,  7667, 11570]])

In [71]:
from sklearn.cluster import KMeans

- k-平均法では、予め分割するクラスター数を決めておく必要がある。

- クラスター数を決定するハイパーパラメータは**n_clusters**になる。

In [72]:
kmeans=KMeans(n_clusters=3, random_state=0)
kmeans.fit(x)

KMeans(n_clusters=3, random_state=0)

- .cluster_center_ で各クラスターの重心(中心の座標)を表示できる。

- 今回はデータセットの次元数が6次元、クラスターの数は3のため、3行6列の値を確認できる。

In [73]:
kmeans.cluster_centers_

array([[ 5043.5483871 ,  3486.41935484,  7017.74193548,  2206.        ,
         2512.90322581,  3509.12903226],
       [28681.25      ,  5637.66666667,  1298.75      ,  1271.        ,
         1770.875     ,   716.95833333],
       [17266.66666667,  2730.93333333, 10721.66666667,   590.86666667,
         3447.33333333,  9851.2       ]])

In [74]:
cluster = kmeans.predict(x)
cluster

array([2, 1, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 0,
       2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1], dtype=int32)

In [75]:
df_cluster=df.copy()
df_cluster['cluster'] = cluster

In [76]:
df_cluster.head()

,No,弁当・麺類,飲料,おにぎり・サンドイッチ,スイーツ,カップスープ,サラダ,cluster
0,1,25350,3650,8945,0,4867,8945,2
1,2,24500,0,0,1827,0,0,1
2,3,23050,5750,11570,0,7667,11570,2
3,4,22850,4100,10145,0,5467,10145,2
4,5,22500,0,0,0,0,0,1


**クラスター結果の考察**

- 予測結果をどう使うかは人間が考慮するべき。


- 今回は、各クラスターの平均購買金額から考察する。

In [77]:
df_results = pd.DataFrame()

In [78]:
df_cluster[df_cluster["cluster"]==0].mean().tolist()

[37.806451612903224,
 5043.548387096775,
 3486.4193548387098,
 7017.741935483871,
 2206.0,
 2512.9032258064517,
 3509.1290322580644,
 0.0]

In [79]:
df_results["cluster0"]=df_cluster[df_cluster["cluster"]==0].mean().tolist()
df_results["cluster1"]=df_cluster[df_cluster["cluster"]==1].mean().tolist()
df_results["cluster2"]=df_cluster[df_cluster["cluster"]==2].mean().tolist()
df_results

,cluster0,cluster1,cluster2
0,37.806452,46.250000,13.533333
1,5043.548387,28681.250000,17266.666667
2,3486.419355,5637.666667,2730.933333
3,7017.741935,1298.750000,10721.666667
4,2206.000000,1271.000000,590.866667
5,2512.903226,1770.875000,3447.333333
6,3509.129032,716.958333,9851.200000
7,0.000000,1.000000,2.000000


In [80]:
df_results=df_results.set_index(df_cluster.columns)
df_results

,cluster0,cluster1,cluster2
No,37.806452,46.250000,13.533333
弁当・麺類,5043.548387,28681.250000,17266.666667
飲料,3486.419355,5637.666667,2730.933333
おにぎり・サンドイッチ,7017.741935,1298.750000,10721.666667
スイーツ,2206.000000,1271.000000,590.866667
カップスープ,2512.903226,1770.875000,3447.333333
サラダ,3509.129032,716.958333,9851.200000
cluster,0.000000,1.000000,2.000000


In [81]:
df_results=df_results.drop(['No', 'cluster']).T
df_results

,弁当・麺類,飲料,おにぎり・サンドイッチ,スイーツ,カップスープ,サラダ
cluster0,5043.548387,3486.419355,7017.741935,2206.000000,2512.903226,3509.129032
cluster1,28681.250000,5637.666667,1298.750000,1271.000000,1770.875000,716.958333
cluster2,17266.666667,2730.933333,10721.666667,590.866667,3447.333333,9851.200000


In [82]:
df_results

,弁当・麺類,飲料,おにぎり・サンドイッチ,スイーツ,カップスープ,サラダ
cluster0,5043.548387,3486.419355,7017.741935,2206.000000,2512.903226,3509.129032
cluster1,28681.250000,5637.666667,1298.750000,1271.000000,1770.875000,716.958333
cluster2,17266.666667,2730.933333,10721.666667,590.866667,3447.333333,9851.200000
